In [ ]:
from ngsolve import *
from netgen.occ import *

from ngsolve.webgui import Draw
from netgen.webgui import Draw as DrawGeo

box = Box((-1,-1,-1), (1,1,0))
sp = Sphere((0.5,0,0), 0.2)
shape = box-sp
geo = OCCGeometry(shape)

h = 0.1
        
mesh = Mesh( geo.GenerateMesh(maxh=h))
mesh.Curve(3)
Draw(mesh);

In [ ]:
order = 4
fes_p = L2(mesh, order=order, all_dofs_together=True)
fes_u = VectorL2(mesh, order=order, piola=True)
fes_tr = FacetFESpace(mesh, order=order)

print ("ndof_p = ", fes_p.ndof, "+", fes_tr.ndof, ", ndof_u =", fes_u.ndof)

traceop = fes_p.TraceOperator(fes_tr, average=True) 

gfu = GridFunction(fes_u)
gfp = GridFunction(fes_p)
gftr = GridFunction(fes_tr)

gfp.Interpolate( exp(-400*(x**2+y**2+z**2)))
gftr.vec.data = traceop * gfp.vec

In [ ]:
p = fes_p.TrialFunction()
v = fes_u.TestFunction()
phat = fes_tr.TrialFunction()

n = specialcf.normal(mesh.dim)

In [ ]:
Bel = BilinearForm(trialspace=fes_p, testspace=fes_u, geom_free = True)
Bel += grad(p)*v * dx -p*(v*n) * dx(element_boundary=True)
Bel.Assemble()

Btr = BilinearForm(trialspace=fes_tr, testspace=fes_u, geom_free = True)
Btr += phat * (v*n) *dx(element_boundary=True)
Btr.Assemble();

In [ ]:
B = Bel.mat + Btr.mat @ traceop

In [ ]:
invmassp = fes_p.Mass(1).Inverse()
invmassu = fes_u.Mass(1).Inverse()

In [ ]:
gfp.Interpolate( exp(-100*(x**2+y**2+z**2)))
gfu.vec[:] = 0

scene = Draw (gftr, draw_vol=False, order=3);

t = 0
dt = 0.5 * h / (order+1)**2 / 2
tend = 0.2

op1 = (dt * invmassu @ B).CreateDeviceMatrix()
op2 = (dt * invmassp @ B.T).CreateDeviceMatrix()

cnt = 0
with TaskManager(): 
    while t < tend:
        t = t+dt
        gfu.vec.data += op1 * gfp.vec
        gfp.vec.data -= op2 * gfu.vec
        cnt = cnt+1
        if cnt%10 == 0:
            gftr.vec.data = traceop * gfp.vec
            scene.Redraw()

## Time-stepping on the device

In [ ]:
try:
    import ngsolve.ngscuda
except:
    pass

In [ ]:
gfp.Interpolate( exp(-100*(x**2+y**2+z**2)))
gfu.vec[:] = 0

scene = Draw (gftr, draw_vol=False, order=3);

t = 0
dt = 0.5 * h / (order+1)**2 / 2
tend = 0.2

op1 = (dt * invmassu @ B).CreateDeviceMatrix()
op2 = (dt * invmassp @ B.T).CreateDeviceMatrix()

devu = gfu.vec.CreateDeviceVector(copy=True)
devp = gfp.vec.CreateDeviceVector(copy=True)

cnt = 0
with TaskManager(): 
    while t < tend:
        t = t+dt
        devu += op1 * devp
        devp -= op2 * devu
        cnt = cnt+1
        if cnt%10 == 0:
            gfp.vec.data = devp
            gftr.vec.data = traceop * gfp.vec
            scene.Redraw()